In [33]:
import sys
sys.path.insert(0,"libs")

In [34]:
answer_section = "Answers" # Heading where answer begins
explanation_section = None # heading where explanation begins
pdf_path = 'mcq_questions.pdf' # path for the pdf of the questions
destination_path = "mcq_question.csv" # path where csv will be saved

In [35]:
import pdfplumber
import re

def extract_paragraphs_from_pdf(pdf_path):
    paragraphs = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            paragraphs.append(text)
        
    return paragraphs

paragraphs = extract_paragraphs_from_pdf(pdf_path)
answer = "".join(paragraphs)
patterNumber =  r'(\d+\.\s)'
patternA  = r'(?=\b[a-d]\.\s)'

paragraph = []
start_indices =[]
end_indices = []

complete_paragraphs = "".join(paragraphs)

if explanation_section!=None and len(complete_paragraphs.split(explanation_section)) > 1 :
    complete_paragraphs = complete_paragraphs.split(explanation_section)
    complete_paragraphs.pop()
    complete_paragraphs = "".join(complete_paragraphs)

complete_paragraphs = complete_paragraphs.split(answer_section)
complete_paragraphs.pop()
complete_paragraphs = "".join(complete_paragraphs)


for pr in re.finditer(patterNumber, complete_paragraphs):
    start_index = pr.start()
    end_index = pr.end()
    start_indices.append(start_index)
    end_indices.append(end_index)

for i in range(1,len(start_indices)-1):
    matched_strng = complete_paragraphs[start_indices[i-1]:start_indices[i]]
    paragraph.append(matched_strng)

paragraph = [re.split(patternA,p) for p in paragraph]

valid = [p for p in paragraph if len(p) == 5 ]
invalid = [p for p in paragraph if len(p) != 5]

answer_parts = answer.split(answer_section)
all_answer = answer_parts[-1].split("\n***")[0]
all_answer = all_answer.replace("\n"," ")

splits :list[str] = all_answer.split(" ")
splits = [ s for s in splits if len(s.strip()) != 0 ]

all_question = []


for v in valid:
    questions : str = v[0].split(".")
    for s in splits:
        s:str
        number = s.split(".")
        if(questions[0].strip() == number[0].strip()):
            correct = 0,
            try:
                if number[1] == "a" : correct = 0
                elif number[1] == "b" : correct = 1
                elif number[1] == "c" : correct = 2
                elif number[1] == "d" : correct = 3
            except IndexError:
                pass
            all_question.append([
                questions[1].strip(),
                v[1].replace("a.","").strip(),
                v[2].replace("b.","").strip(),
                v[3].replace("c.","").strip(),
                v[4].replace("d.","").strip(),
                correct,
                1,
                ""
            ])
            break



In [36]:
import csv

with open(destination_path,mode="w",newline= "\n") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "answer", "answer", "answer", "answer", "correct", "marks", "description"])
    writer.writerows(all_question)